In [110]:
import os

# os.environ["MODIN_ENGINE"] = "ray"
from datetime import datetime, timedelta, timezone

# import modin.pandas as pd
import pandas as pd
from sqlalchemy.orm import Session
from sqlalchemy import select

from utils.db_utils import DB_ENGINE, Category, Restaurant, Item

In [111]:
with Session(DB_ENGINE) as session:
    query = select(Category)
    category_df = pd.read_sql_query(query, session.bind)
    query = select(Restaurant)
    restaurant_df = pd.read_sql_query(query, session.bind)
    query = select(Item)
    item_df = pd.read_sql_query(query, session.bind)
    # cur = db_con.execute("SELECT * FROM items")
    # results = cur.fetchall()
    # UTC is 8 hours ahead of PT
    # pt_now = datetime.now(timezone.utc) - timedelta(hours=8)
    # db_df.to_csv(f"items_{pt_now.isoformat()}.csv")


In [112]:
category_df

,id,name,rel_url
0,1,24 Hours Food,/category/emeryville-ca/24-hours
1,2,Acai food,/category/emeryville-ca/acai
2,3,Afghan Food,/category/emeryville-ca/afghan
3,4,African Food,/category/emeryville-ca/african
4,5,Alcohol,/category/emeryville-ca/alcohol
...,...,...,...
961,962,Wine,/category/oakland-ca/wine
962,963,Wings,/category/oakland-ca/wings
963,964,Wraps,/category/oakland-ca/wrap
964,965,Japanese BBQ,/category/oakland-ca/yakiniku


In [113]:
restaurant_df

,id,name,rating,rel_url,category_id
0,1,Pachas Afghan Kabob House,4.8,/store/pachas-afghan-kabob-house/WwTq4Q9yXbO-p...,196
1,2,Kamdesh Afghan Kabob House,0.0,/store/kamdesh-afghan-kabob-house/2d8xGF3ySHG3...,196
2,3,Yummy Grill/Afghan Kabob House,4.7,/store/yummy-grill-afghan-kabob-house/v3asxWuw...,196
3,4,Afghan Cuisine,0.0,/store/afghan-cuisine/fDNkXRdzVyOrrPxau0UqZg,196
4,5,Abas Hamidi Afghan Cuisine (Concord Blvd),0.0,/store/abas-hamidi-afghan-cuisine-concord-blvd...,196
...,...,...,...,...,...
311,312,Yummy BBQ Kitchen 燒味居燒臘熟餸店,0.0,/store/yummy-bbq-kitchen-%E7%87%92%E5%91%B3%E5...,34
312,313,JP cafe,0.0,/store/jp-cafe/iwEnMzdNQZWH0yU-Da-OPg,34
313,314,D & A Cafe 文記茶餐廳,0.0,/store/d-%26-a-cafe-%E6%96%87%E8%A8%98%E8%8C%B...,34
314,315,Champagne Restaurant,0.0,/store/champagne-restaurant/8N1ZIxaBTyqioXgyko...,34


In [114]:
for _, item in item_df.iterrows():
    if item["name"] is None:
        print(item["rel_url"],"\n")

In [107]:
# def extract_city_from_category_url(row):
#     # ex category_rel_url: /category/emeryville-ca/african
#     return row["category_rel_url"].split("/", 3)[2].rsplit("-", 1)[0].title()


# df["city"] = df.apply(lambda row: extract_city_from_category_url(row), axis=1)
# df.head()

In [108]:
# df.groupby(["city"]).size()

In [109]:
import sqlite3

sqlite3.threadsafety

3